In [1]:
import sys

sys.path.append('../')

import numpy as np
import gym

import or_suite
import pickle

ModuleNotFoundError: No module named 'sklearn_extra'

In [14]:
algorithm = 'Equal_Allocation'
problem = 'default'
dir_path = '../data/allocation_%s_%s'%(algorithm,problem)
dir_path = dir_path+'/trajectory.obj'

In [7]:
print(dir_path)

../data/allocation_Random_default/traj.obj


In [15]:
with open(dir_path, 'rb') as f:
    x = pickle.load(f)


In [16]:
x

[{'iter': 0,
  'episode': 1,
  'step': 0,
  'oldState': (array([100., 100.]), array([30, 48,  5])),
  'action': array([[33.0100138 , 33.0100138 ],
         [33.53376442, 33.53376442],
         [32.96773929, 32.96773929]]),
  'reward': 3.5475933471414356,
  'newState': (array([0.4884825, 0.4884825]), array([37, 43, 12])),
  'info': {'type': array([37, 43, 12])}},
 {'iter': 0,
  'episode': 1,
  'step': 1,
  'oldState': (array([0.4884825, 0.4884825]), array([37, 43, 12])),
  'action': array([[49.60489067, 49.60489067],
         [50.25071796, 50.25071796],
         [48.57770988, 48.57770988]]),
  'reward': 3.99781652586001,
  'newState': (array([-147.94483601, -147.94483601]), array([ 8,  9, 11])),
  'info': {'type': array([ 8,  9, 11])}},
 {'iter': 0,
  'episode': 1,
  'step': 2,
  'oldState': (array([-147.94483601, -147.94483601]), array([ 8,  9, 11])),
  'action': array([[96.74413103, 96.74413103],
         [96.71057309, 96.71057309],
         [96.84671245, 96.84671245]]),
  'reward': 4

In [13]:
# TODO:
#    Figure how to go from dataframe of a trajectory to calculating the fairness metrics